In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import combinations, groupby, chain
from collections import Counter

In [2]:
cluster0_item_rules = pd.read_csv('group0_item_rules.csv', index_col = 0)
cluster1_item_rules = pd.read_csv('group1_item_rules.csv', index_col = 0)
cluster2_item_rules = pd.read_csv('group2_item_rules.csv', index_col = 0)
cluster3_item_rules = pd.read_csv('group3_item_rules.csv', index_col = 0)
cluster4_item_rules = pd.read_csv('group4_item_rules.csv', index_col = 0)
cluster5_item_rules = pd.read_csv('group5_item_rules.csv', index_col = 0)
cluster0_aisle_rules = pd.read_csv('group0_aisle_rules.csv', index_col = 0)
cluster1_aisle_rules = pd.read_csv('group1_aisle_rules.csv', index_col = 0)
cluster2_aisle_rules = pd.read_csv('group2_aisle_rules.csv', index_col = 0)
cluster3_aisle_rules = pd.read_csv('group3_aisle_rules.csv', index_col = 0)
cluster4_aisle_rules = pd.read_csv('group4_aisle_rules.csv', index_col = 0)
cluster5_aisle_rules = pd.read_csv('group5_aisle_rules.csv', index_col = 0)

In [3]:
products = pd.read_csv('products.csv', index_col = 0)
aisles = pd.read_csv('aisles.csv', index_col = 0)
clusters = pd.read_csv('cluster_data', index_col = 0)


In [4]:
cluster_data = clusters.set_index('user_id')
cluster_data


,0,1,2,3,4,5,cluster
user_id,,,,,,,
1,-0.103161,0.048618,-0.031836,-0.005631,-0.053023,-0.055229,3
2,-0.077766,0.077983,-0.076148,-0.101671,0.081781,-0.040407,3
3,0.057496,0.085719,0.001779,-0.013192,-0.028499,0.113056,5
4,-0.055343,0.134731,-0.002394,-0.037706,-0.087809,-0.027360,0
5,0.133936,-0.019591,-0.020960,-0.003730,0.057731,0.138879,5
...,...,...,...,...,...,...,...
206205,-0.021674,0.081783,-0.073480,-0.088434,0.155963,-0.022694,5
206206,-0.040315,-0.036658,-0.024049,0.010062,-0.051017,0.000818,3
206207,-0.022110,-0.032613,-0.009467,-0.015798,0.021492,0.020458,5


In [5]:
cluster_item_rules_dic = {}
cluster_item_rules_dic[0] = cluster0_item_rules
cluster_item_rules_dic[1] = cluster1_item_rules
cluster_item_rules_dic[2] = cluster2_item_rules
cluster_item_rules_dic[3] = cluster3_item_rules
cluster_item_rules_dic[4] = cluster4_item_rules
cluster_item_rules_dic[5] = cluster5_item_rules

In [6]:
cluster_aisle_rules_dic = {}
cluster_aisle_rules_dic[0] = cluster0_aisle_rules
cluster_aisle_rules_dic[1] = cluster1_aisle_rules
cluster_aisle_rules_dic[2] = cluster2_aisle_rules
cluster_aisle_rules_dic[3] = cluster3_aisle_rules
cluster_aisle_rules_dic[4] = cluster4_aisle_rules
cluster_aisle_rules_dic[5] = cluster5_aisle_rules

In [7]:
def product_to_product(cluster, product_id, item_lift, product_name, num_products):
    df = cluster_item_rules_dic[cluster]
    df = df[(df['item_A'] == product_id) | (df['item_B'] == product_id)][['product_name_A','item_A','product_name_B','item_B','confidenceAtoB','lift']].sort_values('lift', ascending = False)
    df = df[df['lift'] > item_lift]
    df = df.sort_values('lift', ascending = False)
    df = df.head(n = num_products)
    product_to_product_associations = df['product_name_A'].values.tolist()
    for x in df['product_name_B'].values.tolist():
        product_to_product_associations.append(x)
    product_to_product_associations = [x for x in product_to_product_associations if x != product_name]
    return product_to_product_associations

In [8]:
def pdp_recommender(user_id, product_id, item_lift, num_products):
    product_name = products.at[product_id,'product_name']
    aisle_id = products.at[product_id,'aisle_id']
    aisle_name = aisles.at[aisle_id,'aisle']
    cluster = cluster_data.at[user_id, 'cluster']
    print(cluster)
    return product_to_product(cluster = cluster, product_id = product_id, item_lift = item_lift, 
                              product_name = product_name, num_products = num_products)

In [9]:
users = [6,26,1,81,4,7]
i = 0
for x in users:
    print('cluster ' + str(i) + str(pdp_recommender(x,39055,1,5)))
    i = i + 1

2
cluster 0['Thin & Light Tortilla Chips', 'Red Peppers', 'Organic Lemon', 'Organic Cucumber', 'Organic Grape Tomatoes']
4
cluster 1[]
3
cluster 2['Real Guacamole', 'Thin & Light Tortilla Chips', 'Original Hummus', 'Organic Reduced Fat 2% Milk', 'Thick & Crispy Tortilla Chips']
1
cluster 3[]
0
cluster 4['Thin & Light Tortilla Chips', 'Organic Raspberries', 'Banana', 'Organic Strawberries', 'Organic Hass Avocado']
5
cluster 5['Thin & Light Tortilla Chips', 'Organic Large Brown Grade AA Cage Free Eggs', 'Organic Reduced Fat 2% Milk', 'Organic Large Grade AA Brown Eggs', 'Thick & Crispy Tortilla Chips']
